In [83]:
import cv2
import numpy as np
import random
import time

In [84]:
random_seed = 2022

In [85]:
#blank_image = np.zeros((20,20,3), np.uint8)
#blank_image[:] = (255,0,0)
#cv2.imwrite("new.png",blank_image)

In [86]:
source_image_path = "lena.png"
source_image = cv2.imread(source_image_path)

In [87]:
text_to_hide = "This is a variable text to hide in an image."
text_array = list(text_to_hide)
text_array = list(map(lambda x: ord(x),text_array))
text_array = np.array(text_array)
text_array.astype(dtype='uint8')



array([ 84, 104, 105, 115,  32, 105, 115,  32,  97,  32, 118,  97, 114,
       105,  97,  98, 108, 101,  32, 116, 101, 120, 116,  32, 116, 111,
        32, 104, 105, 100, 101,  32, 105, 110,  32,  97, 110,  32, 105,
       109,  97, 103, 101,  46], dtype=uint8)

In [88]:
def encode(img_path,text):
  img = cv2.imread(img_path)
  img_height,img_width,_ = img.shape

  text_array = list(text)
  text_array = list(map(lambda x: ord(x),text_array))
  text_array = np.array(text_array)
  text_array.astype(dtype='uint8')

  hide_text_dim = len(text)
  
  # Get random seed
  y_random_indices, x_random_indices = list(range(img_height)), list(range(img_width))
  random.seed(random_seed)
  random.shuffle(x_random_indices)
  random.shuffle(y_random_indices)

  # Injection watermark
  hide_layer = np.zeros(img.shape, dtype=np.uint8)
  for x in range(hide_text_dim):
    hide_layer[y_random_indices[x], x_random_indices[x]] = text_array[x]

  img_freq = np.fft.fft(img)
  encoded_freq = img_freq + hide_layer

  new_img = np.fft.ifft(encoded_freq)
  new_img = np.real(new_img).astype(np.uint8)

  global copy_img
  copy_img = new_img

  #cv2_imshow(new_img)
  cv2.imwrite("encoded.png",new_img)



In [89]:
 encode(source_image_path,text_to_hide)

In [90]:
def decode(src_path,src_encoded):
  img = cv2.imread(src_path)
  img_height,img_width,_ = img.shape

  img_encoded = cv2.imread(src_encoded)
  
  # Get random seed
  y_random_indices, x_random_indices = list(range(img_height)), list(range(img_width))
  random.seed(random_seed)
  random.shuffle(x_random_indices)
  random.shuffle(y_random_indices)

  img_freq = np.fft.fft(img)
  encoded_freq = np.fft.fft(img_encoded)

  hidden_layer = encoded_freq - img_freq

  hidden_layer = np.real(hidden_layer).astype(np.uint8)

  dim_text = 0
  for rows in hidden_layer:
    for elem in rows:
      if elem[0] != 0:
        dim_text += 1

  recreated_text = []
  for i in range(dim_text):
    recreated_text.append("0")
  for i in range(dim_text):
    recreated_text[i] = hidden_layer[y_random_indices[i],x_random_indices[i]][0]

  decoded_text = ''.join(map(lambda x : chr(x),recreated_text))
  print(decoded_text)


In [91]:
encoded_image = "encoded.png"
decode(source_image_path,encoded_image)

This is a variable text to hide in an image.
